# Week 09 - Topic Modeling - Latent Dirichlet Allocation

------

## Part 1

Welcome back! Today, we'll work on a very useful text analysis technique called Topic Modeling. Particularly, we will use its most popular and powerful algorithm: Latent Dirichlet Allocation, or LDA

Before you implement your own LDA on STAN, it is important that you understand all concepts well. In this notebook, we will use an "off-the-shelf" LDA implementation, from scikit-learn (sklearn), and we'll play a little bit with the Dirichlet distribution. Only after that, you'll do your own LDA in STAN. 


We start by the usual imports

In [1]:
import pystan
import pystan_utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Let's import LDA from sklearn.

In order to use it, one needs to convert the documents into a Bag-of-Words representation. The object CountVectorizer does that, so let's import that too

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



Let's try a different kind of dataset, for a change. 

Python has pretty neat packages for data collection, and one that is pretty interesting is the wikipedia package. You can play with that tremendously powerful resource! :-)

First, please install the package.

In [3]:
import wikipedia

Let's try to play with pages that have location labels. We will center our search in Copenhagen

In [4]:
pp=wikipedia.geosearch(55.676098, 12.568337, results=200, radius=10000) # check the Wikipedia Python API package to 
# see the possibilities!

#Or, you could try other kinds of pages... 
#pp=wikipedia.search('norse gods', results=150)  #uncomment this line if you want to try using the search method (we tested with Norse Gods)

You may be curious about what the variable pp  contains. Plese check

In [5]:
pp

['Timeline of Copenhagen',
 'Copenhagen',
 'Rådhuspladsen Station',
 'Dragon Fountain, Copenhagen',
 'City Hall Square, Copenhagen',
 '2000 UEFA Cup Final riots',
 'Copenhagen Fire of 1728',
 "Jens Olsen's World Clock",
 'Industriens Hus',
 'Palace Hotel (Copenhagen)',
 'Lur Blowers',
 'Copenhagen City Hall',
 'Copenhagen Municipality',
 'Vestergade',
 'Grand Theatre (Copenhagen)',
 'H. C. Andersens Boulevard',
 'National Scala',
 'Circus Building, Copenhagen',
 'Pantomimeteatret',
 'Lavendelstræde',
 'Søren Kierkegaard Research Center',
 'Danish Cultural Institute',
 'Tre Hjorter',
 'Copenhagen Waterworks',
 'Axeltorv',
 'Vester Voldgade',
 'Studiestræde',
 'Axelborg',
 'Socialist Youth Front',
 'Tivoli Gardens',
 'Eurovision Song Contest 1964',
 'Copenhagen Central Fire Station',
 'Suhr House',
 'Assessor Bachmann House',
 'Danish Agency for Culture and Palaces',
 'INDEX: Design to Improve Life',
 'Danish Design Centre',
 'Nimb Hotel',
 'Palads Teatret',
 'Danish Arts Foundation',
 '

We created a simple wrapper, that gets the summary content of each page. 

In [6]:

#The idea of this wrapper is to get a content summary for each page. This summary will be a "document" in your LDA
#Sometimes, there is no summary in a wikipedia page, so the call wikipedia.page(title).summary returns an error
#
#In Python, to make sure your program doesn't just stop, you need to use the "try ... except" block

def get_wiki_content(title):
    try:   
        return wikipedia.page(title).summary  
    except:
        pass
    
    return 'NA'   #In case there was an error in the call, you'll get 'NA'
        

Let's get all the text, then

In [7]:

# A set of texts is called a "corpus"
corpus=[get_wiki_content(page) for page in pp]  # this may take a while... 

corpus=list(filter(('NA').__ne__, corpus))  #remove all the 'NA' documents

Take a careful look at the corpus variable

We need to create our Bag-of-Words representation (BoW). Here's how

In [ ]:
vectorizer=CountVectorizer(stop_words='english', max_df=1.0, min_df=0.025) #create a CountVectorizer object. stop_words is a list a of words that are 
                                                 #irrelevant (for example, at, in, on, are, be, have...)


We should add some new stop-words that could make sense in our case... For example, the word "copenhagen" will be everywhere and is not informative at all (if the search is centered in Copenhagen, what should the documents be about?...)! Also, there are "syntactic" words that relate to the webpages collected (e.g. "document", "com", "window"...). 

If we don't clean the texts, our topics will become full of noise (just try commenting the code below and see the results).

**in fact, after seeing your results, you may want to come back here to extend your stop word list...**


In [ ]:

from sklearn.feature_extraction import text 

my_stop_words=text.ENGLISH_STOP_WORDS.union(['copenhagen', 'danish', 'denmark', 'document', 'function', 'id', 'com', 'window', 'true', 'onload', 'app', 'check', 'time', 'adroll', 'parentnode','hereor','getelementsbytagname','script','download', 'search', 'online', 'view','emailprotected', 'emailing', 'calling'])
vectorizer=CountVectorizer(stop_words=my_stop_words, max_df=1.0, min_df=0.04) #create a CountVectorizer object. stop_words is a list a of words that are 
                                                 #irrelevant (for example, at, in, on, are, be, have...)
                                                 


As many other objects in Sklearn, CountVectorizer is applied with the function fit_transform

In [ ]:
descriptions_bow=vectorizer.fit_transform(corpus)   #creates a BoW representation
description_vocabulary = vectorizer.get_feature_names()  #gets the words that correspond to each element of the BoWa

This can be confusing. Check carefully the content of the variables you just created

In [ ]:
print("number of documents is %d"%len(corpus))
print("number of different words is %d"% len(description_vocabulary))
print("Hence, shape of token matrix is", descriptions_bow.get_shape())
print("Second document text:\n", corpus[1])
print("BoW (after stopwords removed):\n", descriptions_bow[1])


It is finally time to run our LDA! We will try with sklearn first...

In [ ]:
lda=LatentDirichletAllocation(n_components=10, learning_method='batch')
x=lda.fit_transform(descriptions_bow)

It's important to understand well both objects, x and lda. Check them carefully... for example, check what methods are available in the lda object. And check the dimensionality of x. What does it mean?

Handy code from Sklearn website:

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

Use it to see your topics. Can you tell the general concept behind each one?

Can you propose the concept "underlying" each topic (or at least some of them):
- Which topics relate to historical buildings?
- Which topics relate to transport?
- Which topics relate to culture?

Do you want to change the parameters (number of topics, values for alpha and beta, stop word list...) and see the changes in the topics? 

In [ ]:
n_top_words=12
print_top_words(lda, description_vocabulary, n_top_words)

--------


## Part 2 - Understanding the dirichlet distribution

The Dirichlet distribution is available as numpy.random.dirichlet(alpha, size=None)

...so, try it! For example, obtain draws from this distribution using different values of alpha.

In [ ]:
print(np.random.dirichlet([.2,.2, .2]))
print(np.random.dirichlet([.1,.1, .9]))
print(np.random.dirichlet([1,1, 1]))


Whenever you can, try to visualize it. Remember what we did in the slides. Try to do the same thing!

**feel free to use the function below, to plot points from a dirichlet distribution, onto a 2D simplex**

In [ ]:
'''Function to plot points in a simplex'''

# Based on post from Thomas Boggs (http://blog.bogatron.net/blog/2014/02/02/visualizing-dirichlet-distributions/)

import matplotlib.tri as tri

_corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
_triangle = tri.Triangulation(_corners[:, 0], _corners[:, 1])

def plot_points(X):
    '''Plots a set of points in the simplex.

    Arguments:

        `X` (ndarray): A 2xN array (if in Cartesian coords) or 3xN array
                       (if in barycentric coords) of points to plot.
    '''
    
    X = X.dot(_corners)  #This is what converts the original points onto the simplex (it projects on it, through dot product)
    plt.plot(X[:, 0], X[:, 1], 'k.', ms=1)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    plt.triplot(_triangle, linewidth=1)

Generate 10000 points from the dirichlet distribution and plot them using the function above

In [ ]:
pts=np.random.dirichlet([.1,.5,.5], size=10000)

In [ ]:
plot_points(pts)

Try with different values of $\alpha$

In [ ]:
plt.figure(figsize=(8, 6))
alphas = [[0.999] * 3,
          [5] * 3,
          [2, 5, 15]]
for (i, alpha) in enumerate(alphas):
    plt.subplot(2, len(alphas), i + 1)
    dist = np.random.dirichlet(alpha, size=5000)
    title = r'$\alpha$ = (%.3f, %.3f, %.3f)' % tuple(alpha)
    plt.title(title, fontdict={'fontsize': 8})
    plot_points(dist)


----------------

## Part 3 - Implement your own LDA model in STAN

It is time to make your own model in STAN! :-) Go for it!

In [ ]:

LDA_STAN="""
data{
    int<lower=1> I;
    int<lower=1> J[I];  // We mean J=||W_i||
    int<lower=2> K;
    int<lower=2> C;
    vector<lower=0>[K] alpha;
    vector<lower=0>[C] beta;
    int<lower=2> MAX_J;
    int W[I,MAX_J];
    }

parameters{
    simplex[K] theta[I];
    simplex[C] phi[K];
}

model{
        
    for (k in 1:K)
        phi[k]~dirichlet(beta);
        
    for (i in 1:I){
        theta[i]~dirichlet(alpha);
        
        for (j in 1:J[i]){
                real gamma[K];
                for (k in 1:K)
                    gamma[k]=log(theta[i,k])+log(phi[k,W[i][j]]);
                
                target+=(log_sum_exp(gamma));
           }
    
    }

}

"""



The data preparation can be a bit tricky, so we did it for you. 

In [ ]:

analyzer=vectorizer.build_analyzer()

K=10    #number of topics
I=len(corpus)               #number of documents
C=len(description_vocabulary)  #number of different words in the dictionary
alpha=np.full(K, 1.0/K)     #alpha and
beta=np.full(C, 1.0/C)      #beta parameters
J=np.zeros(I, dtype='int')   #each document will have a different number of words (we initialize it here, but complete it below)


#The list D will contain a list of documents
D=[]        
for i in range(I):  
    valid_words=[w for w in analyzer(corpus[i]) if w in description_vocabulary]
    d=[description_vocabulary.index(w)+1 for w in valid_words]  #each document is a sequence of words
    if len(d)==0:
        continue    #sometimes it happens that a document becomes empty after cleaning it up...
    D.append(d)
    
I=len(D) #if there were empty documents (due to the data cleaning), we removed them, so we need to decrease I
    
J=[len(d) for d in D]   
MAX_J= max(J)  

#W is the matrix with all words from all documents, to send STAN
W=np.zeros((I, MAX_J), dtype='int')   
W=[np.pad(d, (0, MAX_J-len(d)), 'constant', constant_values=0) for d in D]
W=np.array(W, dtype='int')


Create the "data" dictionary, to send to STAN

In [ ]:
#data={...}
data={'I':I, 'J':J, 'K':K, 'C':C, 'alpha':alpha, 'beta':beta, 'MAX_J':MAX_J, 'W':W}

Run the STAN model now. The VB version will be faster, as usual, and if you're running this in the class, this may be important. 

In [ ]:
## create and run Stan model object
sm = pystan.StanModel(model_code=LDA_STAN)
fit = sm.vb(data=data, iter=10000, algorithm="meanfield", elbo_samples=100, grad_samples=20, seed=42, verbose=True)
#fit = sm.sampling(data=data, iter=1000, chains=4, algorithm="NUTS", verbose=True)
print(fit)


we create a few function to help you inspect the results

In [ ]:

def vb_extract_variable_set(fit, varnames):
    full_names=[name for name in fit['sampler_param_names'] if sum([name.startswith(vname) for vname in varnames])>0]
    l=[]
    for fn in full_names:
        l.append(pystan_utils.vb_extract_variable(fit, fn))
    
    return full_names, l


def extract_topic_VB(fit, K, N, phi='phi'):
    topic=[]
    for n in range(N):
        topic.append(pystan_utils.vb_extract_variable(fit, phi+"."+str(K+1)+"."+str(n+1)))
    return np.array(topic)
    

def print_top_words_VB(model, feature_names, n_top_words, K, phi='phi'):
    N=len(feature_names)
    for k in range(K):
        topic=extract_topic_VB(fit, k, N, phi)
        message = "Topic #%d: " % k
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

    
def print_top_words_STAN(model, feature_names, n_top_words, K):
    for k in range(K):
        topic=np.mean(model.extract('phi')['phi'][:,k,:], axis=0)
        message = "Topic #%d: " % k
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



Please print the top 12 words for your topics. 

In [ ]:
print_top_words_VB(fit, description_vocabulary, 12, K)

-----------

Finally, as just a useful tip, we want to help you save your models after estimating (you don't want to re-run STAN again if you have estimated the model before!). 

The code below serves to save the STAN model and the "fit" variable, which has the results of the inference.

In [ ]:
import pickle
with open("model_fit.pkl", "wb") as f:
    pickle.dump({'model' : sm, 'fit' : fit}, f, protocol=-1)
    # or with a list
    # pickle.dump([model, fit], f, protocol=-1)

The code below lets you load it again later

In [ ]:
import pickle
with open("model_fit.pkl", "rb") as f:
    data_dict = pickle.load(f)
    # or with a list
    # data_list = pickle.load(f)
fit = data_dict['fit']
# fit = data_list[1]